<a href="https://colab.research.google.com/github/restrepo/crossref/blob/master/crossref.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crossref API

In [1]:
import pandas as pd
import getpass
import time

Resources
* API TUTORIAL: https://github.com/CrossRef/rest-api-doc
* JSON Format: https://github.com/Crossref/rest-api-doc/blob/master/api_format.md

## Search by ISSN

In [2]:
email=getpass.getpass('Email:')

Email:········


In [4]:
dfj=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQ2-K4JSl93PMkHaN5MZjlXy5beaZVy3jlOg7YD6tbrFq0pItttWKfKSiAQy7tcRw/pub?gid=508185721&single=true&output=csv')

In [5]:
dfj=dfj.dropna(subset=['Título de la revista']).reset_index(drop=True)

In [6]:
dfj=dfj.fillna('')

In [7]:
djournals=list(
   dfj.apply(lambda row:{'journal':row['Título de la revista']
                      ,'ISSN':{ 'print'     :row['ISSN Impreso']
                               ,'electronic':row['ISSN']
                              }
                     },axis=1).values
)

Get list of Journals ans ISSNs

In [8]:
journals=[ d.get('journal') for d in djournals if d.get('journal')]

issns=[
        d.get('ISSN').get('print') for d in djournals if d.get('ISSN').get('print')
    ]
issns_e=[
        d.get('ISSN').get('electronic') for d in djournals if d.get('ISSN').get('print')
    ]

Loop in ISSNs: both print and electronic

In [12]:
dft=pd.DataFrame()
ISSN=True
sleep=5
for i in range(len(issns)):
#if True:
    url_e=''
    journal=journals[i]
    issn=issns[i]
    issn_e=issns_e[i]
    items=1000
    crf='https://api.crossref.org'
    if ISSN:
        url='{}/journals/{}/works?rows={}&mailto={}'.format(
              crf,issn,items,email)
        if issn_e:
            url_e='{}/journals/{}/works?rows={}&mailto={}'.format(
                 crf,issn_e,items,email)
    else:
        url='{}/works?query.container-title={}&rows={}&mailto={}'.format(
          crf,journal,items,email)
    try:
        df=pd.read_json(url)
    except:
        df=pd.DataFrame()

    df_e=pd.DataFrame()
    if url_e:
        time.sleep(sleep)
        try:
            df_e=pd.read_json(url)
        except:
            pass
        
    if not df.empty:
        dfi=pd.DataFrame( df.loc['items']['message'] )
        dft=(dft.append(dfi,sort=False)).reset_index(drop=True)
        total_results=df.loc['total-results']['message']
        if items>total_results:
            print('Enough rows: all the {} items have been downloaded'.format(total_results))
        else:
            print('Missing items for issn: {}. Increase `rows` value'.format(issn))
            
    if not df_e.empty:
        dfi_e=pd.DataFrame( df.loc['items']['message'] )
        #Check if there are new items:
        dfi_e=dfi_e[dfi_e['ISSN'].apply(lambda l: 
                     issn not in l if isinstance(l,list) else False)
                   ].reset_index(drop=True)
        if not dfi_e.empty:
            dft=(dft.append(dfi_e,sort=False)).reset_index(drop=True)
            total_results=df_e.loc['total-results']['message']
            if items>total_results:
                print('New items in electronic ISSN:')
                print('Enough rows: all the {} items have been downloaded'.format(total_results))
            else:
                print('Missing items for issn: {}. Increase `rows` value'.format(issn_e))
    time.sleep(sleep)

Enough rows: all the 560 items have been downloaded
Enough rows: all the 149 items have been downloaded
Enough rows: all the 191 items have been downloaded
Enough rows: all the 133 items have been downloaded
Enough rows: all the 451 items have been downloaded
Enough rows: all the 134 items have been downloaded
Enough rows: all the 327 items have been downloaded
Enough rows: all the 164 items have been downloaded
Enough rows: all the 145 items have been downloaded
Enough rows: all the 145 items have been downloaded
Enough rows: all the 307 items have been downloaded
Enough rows: all the 140 items have been downloaded
Enough rows: all the 94 items have been downloaded
Enough rows: all the 80 items have been downloaded
Enough rows: all the 104 items have been downloaded
Enough rows: all the 135 items have been downloaded
Enough rows: all the 176 items have been downloaded
Enough rows: all the 732 items have been downloaded
Enough rows: all the 22 items have been downloaded
Enough rows: al

In [13]:
dft.shape

(4281, 43)

In [15]:
dft.to_json('revistas_crossref.json')

In [17]:
dfcr=pd.read_json('revistas_crossref.json')

In [18]:
dfcr.shape

(4281, 43)

In [22]:
dfcr[dfcr.author.astype(str).str.lower().str.contains('orcid')].shape

(599, 43)

In [28]:
dfcr.keys()

Index(['DOI', 'ISSN', 'URL', 'abstract', 'alternative-id', 'assertion',
       'author', 'container-title', 'content-domain', 'created', 'deposited',
       'editor', 'funder', 'indexed', 'is-referenced-by-count', 'issn-type',
       'issue', 'issued', 'journal-issue', 'language', 'license', 'link',
       'member', 'original-title', 'page', 'prefix', 'published-online',
       'published-print', 'publisher', 'reference', 'reference-count',
       'references-count', 'relation', 'score', 'short-container-title',
       'short-title', 'source', 'subject', 'subtitle', 'title', 'type',
       'update-policy', 'volume'],
      dtype='object')

## Searching citations for the DOIS
See: 
* http://opencitations.net/index/coci/api/v1
* https://opencitations.wordpress.com/2020/01/21/coci-jan-2020-release/  

### Operations
The operations that this API implements are:

* `/references/{doi}`: This operation retrieves the citation data for all the references to other works appearing in the bibliographic entity identified by the input DOI.
* `/citations/{doi}`: Search DOIs citing input (cited) DOI: This operation retrieves the citation data for all the references appearing in other works to the bibliographic entity identified by the input DOI.
* `/citation/{oci}`: This operation retrieves the citation data for the citation identified by the input Open Citation Identifier (OCI).
* `/metadata/{dois}`: This operation retrieves the metadata for all the articles identified by the input DOIs.

In [30]:
import requests

### Example:

In [ ]:
operations=['metadata']
for doi in ['10.1103/PhysRevD.99.075008']:
    for operation in operations:
        url='https://opencitations.net/index/coci/api/v1/{}/{}'.format(
           operation,doi)
        r=requests.get(url)
#print last result
r.json()

### Implementation for current DOIs

In [ ]:
citdf=pd.DataFrame()
metdf=pd.DataFrame()
operations=['citations','metadata']
for doi in dfcr.DOI:
    for operation in operations:
        try:
            r=requests.get('https://opencitations.net/index/coci/api/v1/{}/{}'.format(
                   operation,doi))
        except:
            r=None
        time.sleep(3)
        
        if getattr(r, "status_code", None)==200:
            print('citations found for doi: {}'.format(doi))
            if operations=='citations':
                try:
                    citdf=citdf.append(pd.DataFrame(r.json()),sort=False).reset_index(drop=True)
                    citdf.to_json('tmp1.json')
                except:
                    pass
            elif operations=='metadata':
                try:
                    metdf=metdf.append(pd.DataFrame(r.json()),sort=False).reset_index(drop=True)
                    metdf.to_json('tmp2.json')
                except:
                    pass
        else:
            print('wrong status code, waiting 30s')
            time.sleep(30)

citations found for doi: 10.1016/j.estger.2014.05.001
citations found for doi: 10.1016/j.estger.2014.05.001
citations found for doi: 10.1016/j.estger.2014.06.002
citations found for doi: 10.1016/j.estger.2014.06.002
citations found for doi: 10.1016/j.estger.2014.08.005
citations found for doi: 10.1016/j.estger.2014.08.005
citations found for doi: 10.1016/j.estger.2016.02.001
citations found for doi: 10.1016/j.estger.2016.02.001
citations found for doi: 10.17081/eduhum.21.36.3293
citations found for doi: 10.17081/eduhum.21.36.3293
citations found for doi: 10.17081/eduhum.21.37.3410
citations found for doi: 10.17081/eduhum.21.37.3410
citations found for doi: 10.17081/eduhum.21.37.3444
citations found for doi: 10.17081/eduhum.21.37.3444
citations found for doi: 10.17081/eduhum.21.37.3461
citations found for doi: 10.17081/eduhum.21.37.3461
citations found for doi: 10.17081/eduhum.21.37.3365
citations found for doi: 10.17081/eduhum.21.37.3365
citations found for doi: 10.17081/eduhum.21.37.3

In [ ]:
%%bash
#git add Revistas.ipynb revistas_crossref.json
#git commit -am 'cross ref info'
#git push origin master